In [1]:
import pandas as pd
import numpy as np

import nltk
import os
import re

from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import NMF

from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter

import warnings
warnings.filterwarnings("ignore")

word_lm = WordNetLemmatizer()

#### Data: https://archive.ics.uci.edu/ml/datasets/Health+News+in+Twitter
#### using this file everydayhealth.txt

In [2]:
path = 'Health-Tweets'
col_names = ['tw_id', 'date', 'tweet']
df = pd.read_csv(os.path.join(path, 'everydayhealth.txt'), sep="|", names=col_names)
print len(df)
df.head()

3239


,tw_id,date,tweet
0,304596701757464576,Thu Feb 21 14:21:27 +0000 2013,#FastFood Makes Up 11 Percent of #Calories in ...
1,304595191329853441,Thu Feb 21 14:15:27 +0000 2013,"10 snacks to help you lose weight, burn fat, a..."
2,304587659018371072,Thu Feb 21 13:45:31 +0000 2013,10 foods that boost your skin AND slim your wa...
3,304580073380524032,Thu Feb 21 13:15:22 +0000 2013,What a heart attack feels like in women (it's ...
4,304572560270573569,Thu Feb 21 12:45:31 +0000 2013,#McDonalds oatmeal has almost 7 teaspoons of s...


In [3]:
# some preprocessing

def pre_processing(data_frame, txt_clm):
    
    df_txt = data_frame.copy()
    
    # lower case 
    df_txt['lower_txt'] = df_txt[txt_clm].str.lower()
    
    #replace RTs with ''
    df_txt['links_rm'] = df_txt['lower_txt'].apply(lambda v: v.replace('rt', ''))
    
    # remove punctuation
    df_txt['punc_rm'] = df_txt['lower_txt'].str.replace('[^\w\s]','')
    
    # replace hyperlinks with ''
    df_txt['links_rm'] = df_txt['punc_rm'].apply(lambda v: re.sub(r'http\S+', '', v))
    
    # remove stopwords
    df_txt['stopwords_rm'] = df_txt['links_rm'].apply(lambda x: " ".join(x for x in x.split() if x not in ENGLISH_STOP_WORDS))
    
     # lemmatize all the words
    df_txt['lemmatize_text'] = df_txt['stopwords_rm'].apply(lambda v: " ".join([word_lm.lemmatize(i) for i in v.split()]))
    
    df_txt['processed_text'] = df_txt['lemmatize_text'].apply(lambda x: word_tokenize(x))
    
    df_txt['raw_text'] = df_txt['processed_text'].apply(lambda x: " ".join(w for w in x if len(w)> 3))
    
    return df_txt[['tweet', 'raw_text']]

tweets_data = df.copy()
processed_df = pre_processing(tweets_data[['tweet']], 'tweet')
processed_df['raw_text'].tolist()[:10]

[u'fastfood make percent calorie diet',
 u'snack help lose weight burn build muscle',
 u'food boost skin slim waistline',
 u'heart attack feel like woman different gored',
 u'mcdonalds oatmeal teaspoon sugar healthy fast food isnt',
 u'food boost skin slim waistline',
 u'skipping kegels using talcum powder vaginal health mistake youre probably making',
 u'food boost skin slim waistline',
 u'today happier perform random kindness itll make feel better',
 u'depression isnt everybody different face disorder']

In [4]:
# try a groupby of raw text, since preprocessing now we are able to group them
group_text = processed_df.groupby(['raw_text'], as_index=False).count()
group_text = group_text.rename(columns={'tweet': 'count'})
group_text.sort_values('count', ascending=False).head(10)

,raw_text,count
725,everyday health daily digest,42
817,food boost skin slim waistline,24
824,food longer life,22
542,eating habit pack pound,14
1853,soda drinker listen reason kick soda habit,12
1353,meat surprising food thatll spike blood sugar,11
785,fiberrich food diet asap,11
247,calorie today,10
14,adhd characteristic actually huge plus choosin...,10
968,health symptom shouldnt ignore menshealth,9


In [5]:
cnt_vectorizer = CountVectorizer(stop_words='english')
data_vectorized = cnt_vectorizer.fit_transform(processed_df['raw_text'])
cnt_ftrs_names = cnt_vectorizer.get_feature_names()
cnt_ftrs_df = pd.DataFrame(data_vectorized.toarray(), columns=cnt_ftrs_names)
cnt_ftrs_df.head(3)

,1000,1089,11am,11ampst,1200,12noon,13for13,13in2013,1520lbs,1pmet,...,youself,youth,youtube,youve,yummy,zero,zerocalorie,zocor,zucchini,zzzs
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(processed_df['raw_text'])
feature_names = vectorizer.get_feature_names() # num phrases 
feature_df = pd.DataFrame(X.toarray(), columns=feature_names)
feature_df.head(3)

,1000,1089,11am,11ampst,1200,12noon,13for13,13in2013,1520lbs,1pmet,...,youself,youth,youtube,youve,yummy,zero,zerocalorie,zocor,zucchini,zzzs
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# get top n features
feature_array = np.array(feature_names)
tfidf_sorting = np.argsort(X.toarray()).flatten()[::-1]

n = 30
top_n = feature_array[tfidf_sorting][:n]
print top_n

[u'worry' u'sooner' u'highincome' u'sicker' u'country' u'american' u'zzzs'
 u'fightdepression' u'finger' u'finding' u'financial' u'finally'
 u'filtered' u'filling' u'figure' u'fighting' u'fiery' u'fight'
 u'fitandfabliving' u'field' u'fibromyalgia' u'fibrillation' u'fiberrich'
 u'fiber' u'fewer' u'fever' u'feng' u'fend' u'fence' u'fish']


In [8]:
# apply k means cluster, choosing k as 7
true_k = 7
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)
clusters = model.labels_.tolist()
cluster_items = Counter(clusters)
for k in cluster_items:
    print "Cluster {}: {}".format(k, cluster_items[k])

Cluster 0: 1984
Cluster 1: 582
Cluster 2: 230
Cluster 3: 109
Cluster 4: 184
Cluster 5: 105
Cluster 6: 45


In [9]:
#sort cluster centers by proximity to centroid
cl_cols = ['Cluster 0', 'Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4', 'Cluster 5', 'Cluster 6']
cluster_df = pd.DataFrame(columns = cl_cols)

order_centroids = model.cluster_centers_.argsort()[:, ::-1]

for i in range(true_k):
    join_words = []
    for ind in order_centroids[i, :20]:
        join_words.append(feature_names[ind])
    cluster_df["Cluster {}".format(i)] = join_words
    
cluster_df.head(10)

,Cluster 0,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Cluster 5,Cluster 6
0,health,healthtalk,food,psoriasis,risk,like,life
1,weight,everydayhealth,skin,doctor,heart,vegan,longer
2,reason,eatsmartbd,slim,healthtalk,disease,healthtalk,food
3,calorie,join,waistline,parent,healthtalk,plantdietitian,change
4,exercise,today,boost,older,attack,dreenaburton,worst
5,healthy,chat,sugar,baby,rmichlermd,look,time
6,today,question,spike,sexting,higher,body,year
7,daily,follow,blood,acronym,health,healthy,romance
8,really,welcome,best,managing,factor,everydayhealth,boost
9,better,goal,surprising,psoriatic,woman,feel,limitation


In [10]:
topic_cols = ['Topic 0', 'Topic 1', 'Topic 2', 'Topic 3', 'Topic 4', 'Topic 5', 'Topic 6']
topics_df = pd.DataFrame(columns=topic_cols)

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        
        top_words = []
        for i in topic.argsort()[:-no_top_words - 1:-1]:
            top_words.append(feature_names[i])
                             
        topics_df["Topic {}".format(topic_idx)] = top_words
    return topics_df
        
    
        
# Run NMF
no_topics = 7
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(X)

no_top_words = 20
display_topics(nmf, feature_names, no_top_words).head(10)

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6
0,digest,food,healthtalk,weight,calorie,boost,eating
1,everyday,life,join,lose,today,waistline,habit
2,daily,longer,everydayhealth,loss,burn,slim,pound
3,health,change,follow,help,chocolate,skin,pack
4,worst,worst,eatsmartbd,whats,easy,food,healthy
5,snack,sugar,heart,healthy,sneaky,best,soda
6,ignore,spike,chat,trying,just,blood,reason
7,shouldnt,blood,vegan,reason,reason,sugar,complexion
8,2013,asap,question,dieting,skip,plus,drinker
9,symptom,fiberrich,today,losing,tomorrow,metabolism,kick


In [11]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=7, learning_method='online')
lda_model

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=7, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [12]:
lda_output = lda_model.fit_transform(data_vectorized)
display_topics(lda_model, cnt_ftrs_names, no_top_words).head(10)

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6
0,food,healthtalk,study,health,healthtalk,weight,best
1,healthtalk,today,soda,really,healthy,eating,food
2,life,join,health,reason,everydayhealth,calorie,diet
3,boost,love,chocolate,daily,food,exercise,risk
4,youre,jillianmichaels,goal,everyday,eatsmartbd,help,protein
5,health,need,habit,digest,sugar,cancer,doctor
6,menshealth,diabetes,2013,great,surprising,lose,depression
7,heart,everydayhealth,reason,work,dont,diet,child
8,like,good,baby,whats,people,secret,common
9,skin,psoriasis,minute,breakfast,cspi,loss,answer


In [13]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, cnt_vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=            Freq  cluster  topics          x          y
topic                                                  
1      20.578902        1       1   1.951590   0.806206
4      18.274137        1       2 -47.786217  14.778645
0      15.047031        1       3  38.920841  36.893059
5      14.984261        1       4  51.689217 -13.167347
6      11.026853        1       5  14.720409 -49.254395
3      10.674561        1       6 -35.017014 -35.281433
2       9.414255        1       7 -10.817095  50.866272, topic_info=     Category        Freq             Term       Total  loglift  logprob
term                                                                    
1429  Default  732.000000       healthtalk  732.000000  30.0000  30.0000
3424  Default  132.000000           weight  132.000000  29.0000  29.0000
1432  Default  136.000000          healthy  136.000000  28.0000  28.0000
1221  Default  288.000000             food  288.000000  27.0000  27.0000
1788  Default   89.000000             life   89.000000  26.0000  26.0000
969   Default   86.000000           eating   86.000000  25.0000  25.0000
292   Default   83.000000             best   83.000000  24.0000  24.0000
1419  Default  173.000000           health  173.000000  23.0000  23.0000
3193  Default  119.000000            today  119.000000  22.0000  22.0000
441   Default  100.000000          calorie  100.000000  21.0000  21.0000
1065  Default  169.000000   everydayhealth  169.000000  20.0000  20.0000
2524  Default   63.000000           really   63.000000  19.0000  19.0000
1672  Default   81.000000             join   81.000000  18.0000  18.0000
860   Default  119.000000             diet  119.000000  17.0000  17.0000
1856  Default   77.000000             love   77.000000  16.0000  16.0000
3015  Default   49.000000            study   49.000000  15.0000  15.0000
448   Default   61.000000           cancer   61.000000  14.0000  14.0000
1669  Default   73.000000  jillianmichaels   73.000000  13.0000  13.0000
2526  Default   88.000000           reason   88.000000  12.0000  12.0000
1460  Default  109.000000             help  109.000000  11.0000  11.0000
1080  Default   86.000000         exercise   86.000000  10.0000  10.0000
2087  Default   68.000000             need   68.000000   9.0000   9.0000
849   Default   67.000000         diabetes   67.000000   8.0000   8.0000
764   Default   45.000000            daily   45.000000   7.0000   7.0000
356   Default   67.000000            boost   67.000000   6.0000   6.0000
2876  Default   42.000000             soda   42.000000   5.0000   5.0000
1064  Default   44.000000         everyday   44.000000   4.0000   4.0000
1846  Default   52.000000             lose   52.000000   3.0000   3.0000
1325  Default   62.000000             good   62.000000   2.0000   2.0000
2638  Default   78.000000             risk   78.000000   1.0000   1.0000
...       ...         ...              ...         ...      ...      ...
1516   Topic7   10.609465             huge   11.341991   2.2962  -5.1697
1707   Topic7   10.434768             kick   11.166776   2.2951  -5.1863
1531   Topic7   10.293720             idea   11.027080   2.2941  -5.1999
999    Topic7   10.197948        ehyoutube   10.929855   2.2936  -5.2092
3351   Topic7   10.178739        vegetable   10.911314   2.2934  -5.2111
880    Topic7   10.163821           dinner   10.896075   2.2934  -5.2126
1385   Topic7   10.046676           handle   10.779516   2.2925  -5.2242
381    Topic7    9.920799            break   10.652718   2.2918  -5.2368
933    Topic7    9.820557          drinker   10.552120   2.2911  -5.2469
2655   Topic7    9.360131             rule   10.093655   2.2875  -5.2949
2666   Topic7    9.316284             safe   10.049096   2.2872  -5.2996
1323   Topic7    9.173633             gone    9.906684   2.2861  -5.3151
1357   Topic7    8.887427            group    9.621329   2.2836  -5.3468
2391   Topic7    8.746693          prepare    9.478722   2.2826  -5.3627
1609   Topic7    8.407